In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [31]:
base_path_csv = 'tabelas_csv/'
def export_to_csv(host, port, user, password, database, query, csv_name):
    try:
        # Criar a conexão com o banco de dados
        connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_string)

        # Executar a query e carregar os dados em um DataFrame
        print("Conectando ao banco de dados...")
        df = pd.read_sql(query, engine)
        df = df.round({
            'N_ls': 0,          # Inteiros, sem casas decimais
            'N_ts': 0,          # Inteiros, sem casas decimais
            'k': 1,             # 1 casa decimal
            'h_s': 1,           # 1 casa decimal
            't_s': 1,           # 1 casa decimal
            'sigma_u': 2,       # 2 casas decimais
            'w_max': 4,         # 4 casas decimais
            'NUS_2A': 3         # 3 casas decimais
        })

        # Salvar o resultado como um arquivo CSV
        print(f"Salvando os dados no arquivo CSV: {csv_name}...")
        df.to_csv(base_path_csv + csv_name, index=False)

    except Exception as e:
        print(f"Erro: {e}")
    return df


In [34]:
base_path_latex = 'tabelas_latex/'
def export_csv_to_latex(latex_file, df):
    try:
        df = df.round({
            'N_ls': 0,          # Inteiros, sem casas decimais
            'N_ts': 0,          # Inteiros, sem casas decimais
            'k': 1,             # 1 casa decimal
            'h_s': 1,           # 1 casa decimal
            't_s': 1,           # 1 casa decimal
            'sigma_u': 2,       # 2 casas decimais
            'w_max': 4,         # 4 casas decimais
            'NUS_2A': 3         # 3 casas decimais
        })
        # Converter o DataFrame para LaTeX
        latex_table = df.to_latex(index=False)  # Gera a tabela LaTeX como string

        # Salvar a tabela LaTeX em um arquivo .tex
        full_path = base_path_latex + latex_file
        with open(full_path, "w") as f:
            f.write(latex_table)

        print(f"Tabela LaTeX criada com sucesso em: {full_path}")
    except Exception as e:
        print(f"Erro: {e}")


In [11]:
# Configuração dos parâmetros do banco
host='localhost'
port=3306
database='constructal_automate_results'
user='constructal_automate'
password='constructal_design_2024'

In [35]:
query = f'''
SELECT *
FROM (
    SELECT sp.N_ls, sp.N_ts, sp.k, sp.h_s, sp.t_s, epb.sigma_u, epb.w_max, 
           ROUND(epb.sigma_u / 59.68, 3) AS NUS_2A
    FROM constructal_automate_results.cbeb_elastoplasticbuckling epb
    INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis spa 
        ON epb.stiffened_plate_analysis_id = spa.id
    INNER JOIN constructal_automate_results.csg_stiffenedplate sp 
        ON spa.stiffened_plate_id = sp.id
) subquery
WHERE NUS_2A < 1;
'''

base_name = 'estudo_caso_nus_inferiores_referencia'
csv_name = base_name + ".csv"
latex_name = base_name + ".tex"

# Chamar a função para exportar os dados
df = export_to_csv(host, port, user, password, database, query, csv_name)
df

Conectando ao banco de dados...
Salvando os dados no arquivo CSV: estudo_caso_nus_inferiores_referencia.csv...


,N_ls,N_ts,k,h_s,t_s,sigma_u,w_max,NUS_2A
0,2,2,56.2,281.0,5.0,25.57,0.1573,0.428
1,2,3,42.2,211.0,5.0,43.71,0.3346,0.732
2,3,2,48.4,242.0,5.0,30.93,0.2370,0.518
3,3,3,37.6,188.0,5.0,52.58,0.4702,0.881
4,4,2,42.4,212.0,5.0,40.16,0.3663,0.673
5,5,2,37.6,188.0,5.0,52.58,0.5612,0.881


In [36]:
export_csv_to_latex(latex_name, df)

Tabela LaTeX criada com sucesso em: tabelas_latex/estudo_caso_nus_inferiores_referencia.tex
